<a href="https://colab.research.google.com/github/Elbereth-Elentari/Book_recommender/blob/master/Scrape_BUW_catalogue.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install scrapy
!pip3 install bs4

In [2]:
import scrapy
from bs4 import BeautifulSoup
import json
import re
import logging


class JsonWriterPipeline(object):

    def open_spider(self, spider):
        self.file = open('/content/drive/My Drive/Library_catalogue.jl', 'w')

    def close_spider(self, spider):
        self.file.close()

    def process_item(self, item, spider):
        line = json.dumps(dict(item)) + "\n"
        self.file.write(line)
        return item


class BookSpider(scrapy.Spider):
    name = 'books'
    custom_settings = {
        'LOG_LEVEL': logging.WARNING,
        'LOG_FILE': '/content/drive/My Drive/Scrapy2.log',
        'ITEM_PIPELINES': {'__main__.JsonWriterPipeline': 100},
    }


    def start_requests(self):
        #urls = [f'https://chamo.buw.uw.edu.pl/search/query?filter_format=book&filter_lang=pol&filter_lang=eng&filter_loc=10000&filter_loc=10002&sort=dateCreated&pageNumber={page}&theme=system' for page in range(1,74856)]
        with open('/content/drive/My Drive/Scrapy.log', 'r') as old_log:
            for line in old_log:
                if '[scrapy.core.scraper] ERROR: Spider error processing' in line:
                    url = re.search('<GET (.+?)>', line).group(1)
                    yield scrapy.Request(url=url, callback=self.parse)


    def parse(self, response):
        soup = BeautifulSoup(response.body, 'html.parser')
        records = soup.find(class_='records').find_all('div', class_='record')
        
        author, storage, WD_signature, publisher,  = '', '', '', ''
        pages, year = 0, 0
        for record in records:
            if 'Adres wyd.' in record.text and 'Opis fiz.' in record.text:
                title = record.find(class_='title').text
                title = re.sub(r' [=/].+', '', title)
                try: author = record.find(class_='author').text
                except: pass
                if 'BUW Magazyn' in record.text: storage = 'magazyn'

                infos = record.find_all('tr')
                for info in infos:
                    if 'Klasyfikacja WD' in info.text:
                        WD_signature = info.a.text
                    elif 'Adres wyd.' in info.text:
                        publisher_candidates = info.find_all('span')
                        for publisher in publisher_candidates:
                            if ('class' in publisher and publisher['class'] != 'highlight') or 'class' not in publisher:
                                publisher_with_colon = re.search(r".+ : ?([\w .']+),?.*(\d{4})", publisher.text)
                                if publisher_with_colon:
                                    publisher = publisher_with_colon.group(1)
                                    year = int(publisher_with_colon.group(2))
                                else:
                                    publisher = re.search(r".+? ([\w .']+),?.*(\d{4})", publisher.text)
                                    if publisher:
                                        year = int(publisher.group(2))
                                        publisher = publisher.group(1)
                    elif 'Opis fiz.' in info.text:
                        pages = info.find('span').text
                        pages = re.sub(r'\[.+?\]', '', pages)
                        pages = int(re.search(r'\d+', pages).group(0))

                yield {'title': title,
                       'author': author,
                       'storage': storage,
                       'WD_signature': WD_signature,
                       'publisher': publisher,
                       'year': year,
                       'pages': pages}

In [ ]:
!scrapy startproject buw
!cd buw

In [ ]:
from scrapy.crawler import CrawlerProcess
from scrapy.utils.project import get_project_settings

process = CrawlerProcess(get_project_settings())

process.crawl(BookSpider)
process.start()